In [1]:
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [14]:
era_dir = 'C:/Users/User/Desktop/Nazarbayev University/ISSAI Summer 2024/analysis_fancy/raw era5/'
gsmap_dir = 'C:/Users/User/Desktop/Nazarbayev University/ISSAI Summer 2024/analysis_fancy/GSMAP/'
chirps_dir = 'C:/Users/User/Desktop/Nazarbayev University/ISSAI Summer 2024/analysis_fancy/chirps/'
fancy_dir = 'C:/Users/User\Desktop/Nazarbayev University/ISSAI Summer 2024/analysis_fancy/fancy_dataset/'
    

In [19]:
for dataframe in os.listdir(era_dir):
    mydf = pd.DataFrame()
    # loading era5 raw data 
    eradf = pd.read_csv(era_dir + dataframe, parse_dates=['date'])
    mydf['basin_id'] = eradf['basin_id']
    mydf['date'] = eradf['date']
    mydf['prcp_era'] = eradf['prcp'] * 1000  # for some reason ERA computes prcp in m not mm
    mydf['temp_mean'] = eradf['temp_mean'] - 273.15 # from Kelvin to Celcius
    mydf['temp_min'] = eradf['temp_min'] - 273.15
    mydf['temp_max'] = eradf['temp_max'] - 273.15
    mydf['vp1'] = 0.1 * (eradf['dew_mean'] ** -4.9283) * (10 ** (23.5518 + (-2937.4 / eradf['dew_mean']))) # Dr. Vadim provided this formula
    mydf['dew_mean'] = eradf['dew_mean'] - 273.15
    mydf['wind_speed'] = (eradf['u_comp_wind'] ** 2 + eradf['v_comp_wind'] ** 2) ** 0.5 # Dr. Vadim provided this formula
    mydf['vp2'] = 0.6108 * np.exp(17.27 * mydf['dew_mean'] / (mydf['dew_mean'] + 237.3)) # Dr. Vadim provided this formula 
    mydf['srad_joules'] = eradf['srad_joules']

    # print(mydf)
    # print(fancy_dir + dataframe.split('_')[1][:5])
    # break

    mydf[['basin_id', 'date', 'prcp_era', 'temp_mean', 'temp_min', 'temp_max', 'dew_mean', 'wind_speed', 'vp1', 'vp2', 'srad_joules']].to_csv(fancy_dir + dataframe.split('_')[1][:5] + '.csv', index=False)
    print(dataframe.split('_')[1][:5] + ' is DONE')


11001 is DONE
11063 is DONE
11068 is DONE
11077 is DONE
11090 is DONE
11094 is DONE
11108 is DONE
11117 is DONE
11124 is DONE
11126 is DONE
11129 is DONE
11131 is DONE
11137 is DONE
11139 is DONE
11143 is DONE
11146 is DONE
11147 is DONE
11157 is DONE
11160 is DONE
11163 is DONE
11164 is DONE
11170 is DONE
11187 is DONE
11188 is DONE
11189 is DONE
11199 is DONE
11207 is DONE
11219 is DONE
11233 is DONE
11242 is DONE
11272 is DONE
11275 is DONE
11291 is DONE
11293 is DONE
11395 is DONE
11397 is DONE
11421 is DONE
11424 is DONE
11432 is DONE
11433 is DONE
11453 is DONE
11461 is DONE
11468 is DONE
11469 is DONE
11661 is DONE
12001 is DONE
12002 is DONE
12008 is DONE
12029 is DONE
12031 is DONE
12032 is DONE
12072 is DONE
12075 is DONE
12564 is DONE
13002 is DONE
13005 is DONE
13016 is DONE
13029 is DONE
13035 is DONE
13038 is DONE
13048 is DONE
13061 is DONE
13064 is DONE
13090 is DONE
13091 is DONE
13095 is DONE
13105 is DONE
13115 is DONE
13128 is DONE
13142 is DONE
13148 is DONE
13198 

In [48]:
for dataframe in os.listdir(gsmap_dir):
    
    bigdf = pd.read_csv(fancy_dir + dataframe.split()[0], parse_dates=['date'])
    gsdf = pd.read_csv(gsmap_dir + dataframe.split()[0], parse_dates=['date'])

    temp = gsdf[['date', 'prcp']]
    merged_df = pd.merge(bigdf, temp, on='date', how='left')
    merged_df.rename(columns={'prcp': 'prcp_gsmap'}, inplace=True)
    # print(merged_df)
    print('\n')

    path = chirps_dir + dataframe.split()[0]
    if os.path.exists(path):
        chirpsdf = pd.read_csv(path, parse_dates=['date'])
        final_df = pd.merge(merged_df, chirpsdf, on='date',how='left')
        final_df.rename(columns={'precipitation':'prcp_chirps'}, inplace=True) 
        # print(final_df.head())
        final_df[['basin_id', 'date', 'prcp_era', 'prcp_gsmap', 'prcp_chirps', 'temp_mean', 'temp_min', 'temp_max', 'dew_mean', 'wind_speed', 'vp1', 'vp2', 'srad_joules']].to_csv(fancy_dir + dataframe, index=False)
    else :
        merged_df['prcp_chirps'] = np.nan
        merged_df[['basin_id', 'date', 'prcp_era', 'prcp_gsmap', 'prcp_chirps', 'temp_mean', 'temp_min', 'temp_max', 'dew_mean', 'wind_speed', 'vp1', 'vp2', 'srad_joules']].to_csv(fancy_dir + dataframe, index=False)

    print(dataframe.split()[0] + " is DONE")
    



11001.csv is DONE


11063.csv is DONE


11068.csv is DONE


11077.csv is DONE


11090.csv is DONE


11094.csv is DONE


11108.csv is DONE


11117.csv is DONE


11124.csv is DONE


11126.csv is DONE


11129.csv is DONE


11131.csv is DONE


11137.csv is DONE


11139.csv is DONE


11143.csv is DONE


11146.csv is DONE


11147.csv is DONE


11157.csv is DONE


11160.csv is DONE


11163.csv is DONE


11164.csv is DONE


11170.csv is DONE


11187.csv is DONE


11188.csv is DONE


11189.csv is DONE


11199.csv is DONE


11207.csv is DONE


11219.csv is DONE


11233.csv is DONE


11242.csv is DONE


11272.csv is DONE


11275.csv is DONE


11291.csv is DONE


11293.csv is DONE


11395.csv is DONE


11397.csv is DONE


11421.csv is DONE


11424.csv is DONE


11432.csv is DONE


11433.csv is DONE


11453.csv is DONE


11461.csv is DONE


11468.csv is DONE


11469.csv is DONE


11661.csv is DONE


12001.csv is DONE


12002.csv is DONE


12008.csv is DONE


12029.csv is DONE


12031.csv is DONE


In [63]:
from datetime import datetime
import pytz
# import astral 
from astral import LocationInfo
from astral.sun import sun

def daylight(date, lat, lon, timezone_str='Asia/Almaty'):
    # Create a location object with the given latitude and longitude
    location = LocationInfo(latitude=lat, longitude=lon)
    
    # Get the local timezone using the provided timezone string
    timezone = pytz.timezone(timezone_str)
    
    # Combine the date with the minimum time (midnight) and localize it to the timezone
    date_local = timezone.localize(datetime.combine(date, datetime.min.time()))
    
    try:
        # Calculate the sunrise and sunset times for the location and date
        s = sun(location.observer, date=date_local)
        sunrise_local = s['sunrise'].astimezone(timezone)
        sunset_local = s['sunset'].astimezone(timezone)
        
        # Calculate the daylight duration in seconds
        daylight_duration = sunset_local - sunrise_local
        daylight_seconds = daylight_duration.total_seconds()
        
        # Handle edge case where daylight duration is negative
        if daylight_seconds < 0:
            raise ValueError(f"Negative daylight duration: {daylight_seconds} seconds (Sunrise: {sunrise_local}, Sunset: {sunset_local})")
        
    except ValueError:
        # Return -1 in case of an error (e.g., invalid location/date, polar night)
        daylight_seconds = -1
    
    return daylight_seconds



In [ ]:
coordinates_df = pd.read_csv('selected_hydro_stations.csv')
coord = {row['id']: [row['lng'], row['lat']] for _, row in coordinates_df.iterrows()}
print(coord)


def daylight_count(row):
    return daylight(row['date'], coord[row['basin_id']][1], coord[row['basin_id']][0])

for elem in os.listdir(era_dir):
    df = pd.read_csv(era_dir + elem, parse_dates=['date'])
    df['daylight'] = df.apply(daylight_count, axis = 1)
    
    # Replace -1 with NaN
    df['daylight'] = df['daylight'].replace(-1, np.nan)

    # Forward fill NaN values, propagating the previous day's valid value
    df['daylight'] = df['daylight'].fillna(method='ffill')
    
    df['srad'] = df['srad_joules'] / df['daylight']
    # print(df.head())
    # break
    df[['basin_id', 'date', 'prcp', 'temp_mean', 'temp_min', 'temp_max', 'dew_mean', 'u_comp_wind', 'v_comp_wind', 'srad_joules', 'daylight', 'srad']].to_csv(era_dir+elem, index=False)
    print(elem + " is Done")
    

In [84]:
# Copying the srad from CAMELS_KZ dataset
# it is computed from ERA5 Land 
# just realized how stupid this idea was: camels_kz has only 42 basins, while yours has 106 basins. ==> IDIOT! 


# camels_dir = 'C:/Users/User/Desktop/Nazarbayev University/ISSAI Summer 2024/CAMELS_KZ/mean_basin_forcing/'

for dataframe in os.listdir(era_dir):
    final_df = pd.read_csv(fancy_dir + dataframe.split('_')[1], parse_dates=['date'])
    df1 = pd.read_csv(era_dir + dataframe, parse_dates=['date'])
    final_df['srad'] = df1['srad']

    final_df[['basin_id', 'date', 'prcp_era', 'prcp_gsmap', 'prcp_chirps', 'temp_mean', 'temp_min', 'temp_max', 'dew_mean', 'wind_speed', 'vp1', 'vp2', 'srad']].to_csv(fancy_dir + dataframe.split('_')[1], index=False)
    print(dataframe.split('_')[1] + " is DONE")
    


11001.csv is DONE
11063.csv is DONE
11068.csv is DONE
11077.csv is DONE
11090.csv is DONE
11094.csv is DONE
11108.csv is DONE
11117.csv is DONE
11124.csv is DONE
11126.csv is DONE
11129.csv is DONE
11131.csv is DONE
11137.csv is DONE
11139.csv is DONE
11143.csv is DONE
11146.csv is DONE
11147.csv is DONE
11157.csv is DONE
11160.csv is DONE
11163.csv is DONE
11164.csv is DONE
11170.csv is DONE
11187.csv is DONE
11188.csv is DONE
11189.csv is DONE
11199.csv is DONE
11207.csv is DONE
11219.csv is DONE
11233.csv is DONE
11242.csv is DONE
11272.csv is DONE
11275.csv is DONE
11291.csv is DONE
11293.csv is DONE
11395.csv is DONE
11397.csv is DONE
11421.csv is DONE
11424.csv is DONE
11432.csv is DONE
11433.csv is DONE
11453.csv is DONE
11461.csv is DONE
11468.csv is DONE
11469.csv is DONE
11661.csv is DONE
12001.csv is DONE
12002.csv is DONE
12008.csv is DONE
12029.csv is DONE
12031.csv is DONE
12032.csv is DONE
12072.csv is DONE
12075.csv is DONE
12564.csv is DONE
13002.csv is DONE
13005.csv 

In [17]:
mswep_dir = 'C:/Users/User/Desktop/Nazarbayev University/ISSAI Summer 2024/analysis_fancy/mswep/'
f_dir = 'C:/Users/User/Desktop/Nazarbayev University/ISSAI Summer 2024/analysis_fancy/fancy_dataset/fancy_meteo/'

for csv in os.listdir(mswep_dir):
    if (csv.split('_')[0] == '11001'):
        continue
    ms_df = pd.read_csv(mswep_dir + csv)
    fancy_df = pd.read_csv(f_dir + csv.split('_')[0] + '.csv')
    # print(ms_df.tail())
    # print(fancy_df.tail())
    # break
    fancy_df['prcp_mswep'] = ms_df['AvgPrecip']
    # print(fancy_df.tail())
    fancy_df.rename(columns={'temp_mean': 't_mean', 'temp_min': 't_min', 'temp_max': 't_max'}, inplace=True)
    # print(fancy_df.head())
    # break
    fancy_df[['basin_id', 'date', 'prcp_era', 'prcp_mswep', 'prcp_gsmap', 'prcp_chirps', 't_mean', 't_min', 't_max', 'dew_mean', 'wind_speed', 'vp1', 'vp2', 'srad']].to_csv(f_dir + csv.split('_')[0] + '.csv', index=False)
    print(csv.split('_')[0] + " is DONE")
    
    

11063 is DONE
11068 is DONE
11077 is DONE
11090 is DONE
11094 is DONE
11108 is DONE
11117 is DONE
11124 is DONE
11126 is DONE
11129 is DONE
11131 is DONE
11137 is DONE
11139 is DONE
11143 is DONE
11146 is DONE
11147 is DONE
11157 is DONE
11160 is DONE
11163 is DONE
11164 is DONE
11170 is DONE
11187 is DONE
11188 is DONE
11189 is DONE
11199 is DONE
11207 is DONE
11219 is DONE
11233 is DONE
11242 is DONE
11272 is DONE
11275 is DONE
11291 is DONE
11293 is DONE
11395 is DONE
11397 is DONE
11421 is DONE
11424 is DONE
11432 is DONE
11433 is DONE
11453 is DONE
11461 is DONE
11468 is DONE
11469 is DONE
11661 is DONE
12001 is DONE
12002 is DONE
12008 is DONE
12029 is DONE
12031 is DONE
12032 is DONE
12072 is DONE
12075 is DONE
12564 is DONE
13002 is DONE
13005 is DONE
13016 is DONE
13029 is DONE
13035 is DONE
13038 is DONE
13048 is DONE
13061 is DONE
13064 is DONE
13090 is DONE
13091 is DONE
13095 is DONE
13105 is DONE
13115 is DONE
13128 is DONE
13142 is DONE
13148 is DONE
13198 is DONE
13201 